# Stock Sentiment Analysis

### 1 - Stock Price increased
### 0 - Stock Price decreased or stayed same

In [10]:
import nltk
import pandas as pd
import sklearn
import numpy as np

In [5]:
dataframe = pd.read_csv('/content/drive/MyDrive/Datasets/stock_dataset.csv',encoding = "ISO-8859-1")
dataframe.head()


,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2000-01-03,0,A 'hindrance to operations': extracts from the...,Scorecard,Hughes' instant hit buoys Blues,Jack gets his skates on at ice-cold Alex,Chaos as Maracana builds up for United,Depleted Leicester prevail as Elliott spoils E...,Hungry Spurs sense rich pickings,Gunners so wide of an easy target,...,Flintoff injury piles on woe for England,Hunters threaten Jospin with new battle of the...,Kohl's successor drawn into scandal,The difference between men and women,"Sara Denver, nurse turned solicitor",Diana's landmine crusade put Tories in a panic,Yeltsin's resignation caught opposition flat-f...,Russian roulette,Sold out,Recovering a title
1,2000-01-04,0,Scorecard,The best lake scene,Leader: German sleaze inquiry,"Cheerio, boyo",The main recommendations,Has Cubie killed fees?,Has Cubie killed fees?,Has Cubie killed fees?,...,On the critical list,The timing of their lives,Dear doctor,Irish court halts IRA man's extradition to Nor...,Burundi peace initiative fades after rebels re...,PE points the way forward to the ECB,Campaigners keep up pressure on Nazi war crime...,Jane Ratcliffe,Yet more things you wouldn't know without the ...,Millennium bug fails to bite
2,2000-01-05,0,Coventry caught on counter by Flo,United's rivals on the road to Rio,Thatcher issues defence before trial by video,Police help Smith lay down the law at Everton,Tale of Trautmann bears two more retellings,England on the rack,Pakistan retaliate with call for video of Walsh,Cullinan continues his Cape monopoly,...,South Melbourne (Australia),Necaxa (Mexico),Real Madrid (Spain),Raja Casablanca (Morocco),Corinthians (Brazil),Tony's pet project,Al Nassr (Saudi Arabia),Ideal Holmes show,Pinochet leaves hospital after tests,Useful links
3,2000-01-06,1,Pilgrim knows how to progress,Thatcher facing ban,McIlroy calls for Irish fighting spirit,Leicester bin stadium blueprint,United braced for Mexican wave,"Auntie back in fashion, even if the dress look...",Shoaib appeal goes to the top,Hussain hurt by 'shambles' but lays blame on e...,...,Putin admits Yeltsin quit to give him a head s...,BBC worst hit as digital TV begins to bite,How much can you pay for...,Christmas glitches,"Upending a table, Chopping a line and Scoring ...","Scientific evidence 'unreliable', defence claims",Fusco wins judicial review in extradition case,Rebels thwart Russian advance,Blair orders shake-up of failing NHS,Lessons of law's hard heart
4,2000-01-07,1,Hitches and Horlocks,Beckham off but United survive,Breast cancer screening,Alan Parker,Guardian readers: are you all whingers?,Hollywood Beyond,Ashes and diamonds,Whingers - a formidable minority,...,Most everywhere: UDIs,Most wanted: Chloe lunettes,Return of the cane 'completely off the agenda',From Sleepy Hollow to Greeneland,Blunkett outlines vision for over 11s,"Embattled Dobson attacks 'play now, pay later'...",Doom and the Dome,What is the north-south divide?,Aitken released from jail,Gone aloft


In [13]:
len(dataframe)

4101

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# New Section

In [14]:
train_data = dataframe[dataframe['Date'] < '20150101']
test_data = dataframe[dataframe['Date'] > '20141231']

In [15]:
# Data Preprocessing & Cleaning

# Removing Punctuations

train_data_cleaned = train_data.iloc[:,2:27]
train_data_cleaned.replace("[^a-zA-Z]", " " ,regex=True,inplace=True)

# Renaming Columns

train_data_cleaned.columns = [str(i) for i in range(25)]


In [16]:
# Converting into lowercase

for i in range(25):
    train_data_cleaned[str(i)] = train_data_cleaned[str(i)].str.lower()
train_data_cleaned.head(1)


,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,a hindrance to operations extracts from the...,scorecard,hughes instant hit buoys blues,jack gets his skates on at ice cold alex,chaos as maracana builds up for united,depleted leicester prevail as elliott spoils e...,hungry spurs sense rich pickings,gunners so wide of an easy target,derby raise a glass to strupar s debut double,southgate strikes leeds pay the penalty,...,flintoff injury piles on woe for england,hunters threaten jospin with new battle of the...,kohl s successor drawn into scandal,the difference between men and women,sara denver nurse turned solicitor,diana s landmine crusade put tories in a panic,yeltsin s resignation caught opposition flat f...,russian roulette,sold out,recovering a title


In [17]:
# Converting each index into a single sentence & appending it into a list then applying Bag of Words Model

headlines=[]
for row in range(0,len(train_data_cleaned.index)):
    headlines.append(' '.join(str(x) for x in train_data_cleaned.iloc[row,0:25]))


In [18]:
# Bag of Words Model

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

cv = CountVectorizer(ngram_range = (2,2))
train_data_modeled = cv.fit_transform(headlines)


In [19]:
#  Random Forest Classifier

rc = RandomForestClassifier(n_estimators=200,criterion='entropy')
rc.fit(train_data_modeled,train_data["Label"])


RandomForestClassifier(criterion='entropy', n_estimators=200)

In [20]:
# Predicting for test dataset

test_transform=[]
for row in range(0,len(test_data.index)):
    test_transform.append(' '.join(str(x) for x in test_data.iloc[row,2:27]))

test_data_modeled = cv.transform(test_transform)
prediction = rc.predict(test_data_modeled)


In [21]:
# Importing sklearn libraries for checking accuracy, confusion matrix, classification report

from sklearn.metrics import classification_report,confusion_matrix,accuracy_score


In [22]:
# Classification Report

Classification_report = classification_report(test_data['Label'],prediction)
print(Classification_report)

# Confusion Matrix

Confusion_matrix = confusion_matrix(test_data['Label'],prediction)
print(Confusion_matrix)

# Accuracy Score

Accuracy = accuracy_score(test_data['Label'],prediction)
print(Accuracy)


              precision    recall  f1-score   support

           0       0.95      0.75      0.84       186
           1       0.80      0.96      0.87       192

    accuracy                           0.86       378
   macro avg       0.87      0.86      0.85       378
weighted avg       0.87      0.86      0.86       378

[[139  47]
 [  7 185]]
0.8571428571428571
